# Periods tutorial

#### Sections
* [Period Parameters](#Period-Parameters)
* [`start` and `end`](#start-and-end)
    * [As dates (including the **Golden Rule**)](#As-dates)
    * [As times](#As-times)
        * [Timezones (`tzin`)](#Timezones-(tzin))
    * [As dates and times](#As-dates-and-times)
    * [Default values](#Default-values)
* [Duration](#Duration)
    * [Trading time (Including the **Silver Rule**)](#Trading-time)
    * [Trading sessions](#Trading-sessions)
    * [Calendar time](#Calendar-time)
* [Invalid period parameters combinations](#Invalid-period-parameters-combinations)
* [Multiple exchanges (`lead_symbol`)](#Multiple-exchanges-(lead_symbol))
    * [Default calendar](#Default-calendar)
    * [`tzin`](#tzin)
* [`add_a_row`](#add_a_row)

#### Notes
* The cell **outputs** shown in this tutorial are based on executing the cells at **2022-05-12 17:32 New York time** (21:32 UTC). Simply rerun the cells to bring any dynamic output up to date.

## Setup

Run the following cell to import tutorial dependencies.

In [2]:
from market_prices import PricesYahoo
import pytz
import pandas as pd
from market_prices.support import tutorial_helpers as th

Run the following cell to instantiate prices objects and define values used in the first part of this tutorial.

In [3]:
prices = PricesYahoo("MSFT")  # prices for US stock Microsoft
xnys = prices.calendar_default
start_T2, end_T2 = th.get_sessions_range_for_bi(prices, prices.bis.T2)
start_T1, end_T1 = th.get_sessions_range_for_bi(prices, prices.bis.T1)
_regular_session_length = pd.Timedelta(hours=6, minutes=30)
start_reg_T1, _, end_reg_T1 = th.get_conforming_cc_sessions(
    prices.cc, _regular_session_length, start_T1, end_T1, 3
)
one_min = pd.Timedelta(1, "T")

## Period Parameters

The parameters that define a period are collectively referred to as 'period parameters'.

The period for which prices are requested can be defined as either the [period from `start` to `end`](#start-and-end) or as a [duration](#Duration) which can be optionally bound with either `start` or `end`.

Durations can be defined in terms of either [trading time](#Trading-time) (`hours` and `minutes`), [trading sessions](#Trading-sessions) (`days`) or [calendar time](#Calendar-time) (`weeks`, `months` and `years`).

Period parameters that define duration in different terms [cannot be combined](#Invalid-period-parameters-combinations).

The [`add_a_row`](#add_a_row) parameter can be used to include the row immediately prior to the start of the evaluated period.

## `start` and `end`

`start` and `end` can be passed to define the period bounds. These bounds can be described as dates or times.

### As dates

When `start` and/or `end` is a date that represents a session then the period will be inclusive of the session.

In [4]:
prices.get("1D", start="2021-12-03", end="2021-12-13")

symbol            MSFT                                              
                  open        high         low       close    volume
2021-12-03  331.989990  332.700012  318.029999  323.010010  41779300
2021-12-06  323.950012  327.450012  319.230011  326.190002  30032600
2021-12-07  331.640015  335.799988  330.100006  334.920013  31021900
2021-12-08  335.309998  335.500000  330.799988  334.970001  24761000
2021-12-09  334.410004  336.489990  332.119995  333.100006  22214200
2021-12-10  334.980011  343.000000  334.790009  342.540009  38095700
2021-12-13  340.679993  343.790009  339.079987  339.399994  28899400

If `start` does not represent a session then the period will start with the first session following `start`. Similarly, if `end` does not represent a session then the period will end with the closest session that preceeds `end`. This observes the **Golden Rule** when evaluating the period - **price data will never be included for any time or date that falls before `start` or after `end`**. 

In [5]:
prices.get("1D", start="2021-12-04", end="2021-12-11")

symbol            MSFT                                              
                  open        high         low       close    volume
2021-12-06  323.950012  327.450012  319.230011  326.190002  30032600
2021-12-07  331.640015  335.799988  330.100006  334.920013  31021900
2021-12-08  335.309998  335.500000  330.799988  334.970001  24761000
2021-12-09  334.410004  336.489990  332.119995  333.100006  22214200
2021-12-10  334.980011  343.000000  334.790009  342.540009  38095700

If the interval is intraday then the period will start at the open of the start session and end at the close of the end session.

In [6]:
# define start and end as sessions for which intraday data available
start, end = start_T2, end_T2
start, end

(Timestamp('2022-03-31 00:00:00', freq='C'),
 Timestamp('2022-04-12 00:00:00', freq='C'))

In [7]:
df = prices.get("5T", start=start, end=end)
df

symbol                                            MSFT              \
                                                  open        high   
[2022-03-31 09:30:00, 2022-03-31 09:35:00)  313.899994  315.140015   
[2022-03-31 09:35:00, 2022-03-31 09:40:00)  312.579987  312.625000   
[2022-03-31 09:40:00, 2022-03-31 09:45:00)  312.109985  312.515015   
[2022-03-31 09:45:00, 2022-03-31 09:50:00)  312.240387  312.500000   
[2022-03-31 09:50:00, 2022-03-31 09:55:00)  311.899994  312.179993   
...                                                ...         ...   
[2022-04-12 15:35:00, 2022-04-12 15:40:00)  281.170013  281.920013   
[2022-04-12 15:40:00, 2022-04-12 15:45:00)  281.769989  282.070007   
[2022-04-12 15:45:00, 2022-04-12 15:50:00)  281.570007  282.500000   
[2022-04-12 15:50:00, 2022-04-12 15:55:00)  282.190002  282.450012   
[2022-04-12 15:55:00, 2022-04-12 16:00:00)  282.489990  282.579987   

symbol                                                                       
                                                   low       close   volume  
[2022-03-31 09:30:00, 2022-03-31 09:35:00)  312.589996  312.589996  1051661  
[2022-03-31 09:35:00, 2022-03-31 09:40:00)  311.299988  312.005005   654932  
[2022-03-31 09:40:00, 2022-03-31 09:45:00)  311.350006  312.191895   372759  
[2022-03-31 09:45:00, 2022-03-31 09:50:00)  311.290009  311.940002   362373  
[2022-03-31 09:50:00, 2022-03-31 09:55:00)  311.519989  312.160004   317484  
...                                                ...         ...      ...  
[2022-04-12 15:35:00, 2022-04-12 15:40:00)  281.029999  281.779999   435454  
[2022-04-12 15:40:00, 2022-04-12 15:45:00)  281.390015  281.589996   449367  
[2022-04-12 15:45:00, 2022-04-12 15:50:00)  281.359985  282.274994   556228  
[2022-04-12 15:50:00, 2022-04-12 15:55:00)  281.750000  282.440002   635292  
[2022-04-12 15:55:00, 2022-04-12 16:00:00)  281.910004  281.980011   977844  

[702 rows x 5 columns]

In [8]:
# let's check...
open_ = xnys.session_open(start).tz_convert(prices.tz_default)
close = xnys.session_close(end).tz_convert(prices.tz_default)
open_, close

(Timestamp('2022-03-31 09:30:00-0400', tz='America/New_York'),
 Timestamp('2022-04-12 16:00:00-0400', tz='America/New_York'))

In [9]:
(open_ == df.index[0].left) and (close == df.index[-1].right)

True

`start` and `end` can be passed as a pandas `Timestamp` or any object that is acceptable as a single-argument input to `pd.Timestamp`.

To represent a date or a session the `pd.Timestamp` must:
* not include a time component (any time component should be 00:00).
* be timezone naive.  

(To the contrary the input will be interpreted as a [time](#Times).)

In [10]:
start = pd.Timestamp("2021-12-07")
end=pd.Timestamp("2021-12-09")
prices.get("1D", start=start, end=end)

symbol            MSFT                                              
                  open        high         low       close    volume
2021-12-07  331.640015  335.799988  330.100006  334.920013  31021900
2021-12-08  335.309998  335.500000  330.799988  334.970001  24761000
2021-12-09  334.410004  336.489990  332.119995  333.100006  22214200

Rather than passing `start` and `end` as kwargs they can be passed positionally as, respectively, the second and third args.

In [11]:
start = pd.Timestamp("2021")
end = pd.Timestamp("2022-02")
prices.get("1D", start, end)

symbol            MSFT                                              
                  open        high         low       close    volume
2021-01-04  222.529999  223.000000  214.809998  217.690002  37130100
2021-01-05  217.259995  218.520004  215.699997  217.899994  23823000
2021-01-06  212.169998  216.490005  211.940002  212.250000  35930700
2021-01-07  214.039993  219.339996  213.710007  218.289993  27694500
2021-01-08  218.679993  220.580002  217.029999  219.619995  22956200
...                ...         ...         ...         ...       ...
2022-01-26  307.989990  308.500000  293.029999  296.709991  90428900
2022-01-27  302.660004  307.299988  297.929993  299.839996  53481300
2022-01-28  300.230011  308.480011  294.450012  308.260010  49743700
2022-01-31  308.950012  312.380005  306.369995  310.980011  46444500
2022-02-01  310.410004  310.630005  305.130005  308.760010  40950400

[273 rows x 5 columns]

Note how `start` and `end` are internally passed on to `pd.Timestamp`...

In [12]:
prices.get("1D", "2021", "2022-02")

symbol            MSFT                                              
                  open        high         low       close    volume
2021-01-04  222.529999  223.000000  214.809998  217.690002  37130100
2021-01-05  217.259995  218.520004  215.699997  217.899994  23823000
2021-01-06  212.169998  216.490005  211.940002  212.250000  35930700
2021-01-07  214.039993  219.339996  213.710007  218.289993  27694500
2021-01-08  218.679993  220.580002  217.029999  219.619995  22956200
...                ...         ...         ...         ...       ...
2022-01-26  307.989990  308.500000  293.029999  296.709991  90428900
2022-01-27  302.660004  307.299988  297.929993  299.839996  53481300
2022-01-28  300.230011  308.480011  294.450012  308.260010  49743700
2022-01-31  308.950012  312.380005  306.369995  310.980011  46444500
2022-02-01  310.410004  310.630005  305.130005  308.760010  40950400

[273 rows x 5 columns]

Consider the following...

In [13]:
prices.get("2M", "2021", "2021-06-30")

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-01-01, 2021-03-01)  222.529999  246.130005  211.940002  232.380005   
[2021-03-01, 2021-05-01)  235.899994  263.190002  224.259995  252.179993   
[2021-05-01, 2021-07-01)  253.399994  271.649994  238.070007  270.899994   

symbol                                  
                                volume  
[2021-01-01, 2021-03-01)  1.139039e+09  
[2021-03-01, 2021-05-01)  1.293607e+09  
[2021-05-01, 2021-07-01)  1.003657e+09

Hold on, the last indice there runs to 2021-07-01, although that's after `end` (2021-06-30) and the golden rules states that the returned prices will never include any data that falls after the `end`! What's going on!?

Recall from the [intervals](./intervals.ipynb) tutorial that indices expressed as intervals are always closed on the left, such that the interval here includes all sessions from and including 2021-05-01, to but NOT including 2021-07-01.

### As times

Alternatively `start` and `end` can be passed as times. 

In [14]:
prices.get("1D", start="2021-12-01 09:30", end="2021-12-07 16:00")

symbol            MSFT                                              
                  open        high         low       close    volume
2021-12-01  335.130005  339.279999  329.390015  330.079987  33337600
2021-12-02  330.299988  333.489990  327.799988  329.489990  30766000
2021-12-03  331.989990  332.700012  318.029999  323.010010  41779300
2021-12-06  323.950012  327.450012  319.230011  326.190002  30032600
2021-12-07  331.640015  335.799988  330.100006  334.920013  31021900

The `start` and `end` above represent the open and close of their respective sessions...

In [15]:
(
    xnys.session_open("2021-12-01").tz_convert(prices.tz_default),
    xnys.session_close("2021-12-07").tz_convert(prices.tz_default)
)

(Timestamp('2021-12-01 09:30:00-0500', tz='America/New_York'),
 Timestamp('2021-12-07 16:00:00-0500', tz='America/New_York'))

Look what happens if the `start` and `end` are changed to one minute 'inside' of their respective session's bounds...

In [16]:
prices.get("1D", "2021-12-01 09:31", "2021-12-07 15:59")

symbol            MSFT                                              
                  open        high         low       close    volume
2021-12-02  330.299988  333.489990  327.799988  329.489990  30766000
2021-12-03  331.989990  332.700012  318.029999  323.010010  41779300
2021-12-06  323.950012  327.450012  319.230011  326.190002  30032600

The 2021-12-01 and 2021-12-07 sessions are no longer included. If these sessions were to be included then the start of the data would include the minute 2021-12-01 09:30 through 09:31 and the end of the data would include the minute 2021-12-07 15:59 through 16:00. The exclusion of these sessions is the **Golden Rule** in action - **price data will never be included for any time or date that falls before `start` or after `end`**.

The same principle can be shown with intraday 5 minute data.

In [17]:
# using sessions for which intraday data is available...
start_session, end_session = start_T2, xnys.next_session(start_T2)
start, end = xnys.session_open(start_session), xnys.session_close(end_session)
start, end = start.astimezone(prices.tz_default), end.astimezone(prices.tz_default)
start, end

(Timestamp('2022-03-31 09:30:00-0400', tz='America/New_York'),
 Timestamp('2022-04-01 16:00:00-0400', tz='America/New_York'))

In [18]:
prices.get("5T", start, end)

symbol                                            MSFT              \
                                                  open        high   
[2022-03-31 09:30:00, 2022-03-31 09:35:00)  313.899994  315.140015   
[2022-03-31 09:35:00, 2022-03-31 09:40:00)  312.579987  312.625000   
[2022-03-31 09:40:00, 2022-03-31 09:45:00)  312.109985  312.515015   
[2022-03-31 09:45:00, 2022-03-31 09:50:00)  312.240387  312.500000   
[2022-03-31 09:50:00, 2022-03-31 09:55:00)  311.899994  312.179993   
...                                                ...         ...   
[2022-04-01 15:35:00, 2022-04-01 15:40:00)  306.429993  306.565002   
[2022-04-01 15:40:00, 2022-04-01 15:45:00)  306.299988  306.679993   
[2022-04-01 15:45:00, 2022-04-01 15:50:00)  306.100006  306.635010   
[2022-04-01 15:50:00, 2022-04-01 15:55:00)  306.670013  309.000000   
[2022-04-01 15:55:00, 2022-04-01 16:00:00)  308.630005  309.619995   

symbol                                                                       
                                                   low       close   volume  
[2022-03-31 09:30:00, 2022-03-31 09:35:00)  312.589996  312.589996  1051661  
[2022-03-31 09:35:00, 2022-03-31 09:40:00)  311.299988  312.005005   654932  
[2022-03-31 09:40:00, 2022-03-31 09:45:00)  311.350006  312.191895   372759  
[2022-03-31 09:45:00, 2022-03-31 09:50:00)  311.290009  311.940002   362373  
[2022-03-31 09:50:00, 2022-03-31 09:55:00)  311.519989  312.160004   317484  
...                                                ...         ...      ...  
[2022-04-01 15:35:00, 2022-04-01 15:40:00)  306.209991  306.329987   241517  
[2022-04-01 15:40:00, 2022-04-01 15:45:00)  305.940002  306.114990   367177  
[2022-04-01 15:45:00, 2022-04-01 15:50:00)  305.851013  306.635010   396375  
[2022-04-01 15:50:00, 2022-04-01 15:55:00)  306.670013  308.665009  1101768  
[2022-04-01 15:55:00, 2022-04-01 16:00:00)  308.519989  309.420013  1636448  

[156 rows x 5 columns]

Above `start` and `end` represent, respectively, the open and close of two contiguous sessions. Look what happens if `start` is passed as one minute after the session open.

In [19]:
prices.get("5T", start + one_min, end)

symbol                                            MSFT              \
                                                  open        high   
[2022-03-31 09:35:00, 2022-03-31 09:40:00)  312.579987  312.625000   
[2022-03-31 09:40:00, 2022-03-31 09:45:00)  312.109985  312.515015   
[2022-03-31 09:45:00, 2022-03-31 09:50:00)  312.240387  312.500000   
[2022-03-31 09:50:00, 2022-03-31 09:55:00)  311.899994  312.179993   
[2022-03-31 09:55:00, 2022-03-31 10:00:00)  312.170013  312.309998   
...                                                ...         ...   
[2022-04-01 15:35:00, 2022-04-01 15:40:00)  306.429993  306.565002   
[2022-04-01 15:40:00, 2022-04-01 15:45:00)  306.299988  306.679993   
[2022-04-01 15:45:00, 2022-04-01 15:50:00)  306.100006  306.635010   
[2022-04-01 15:50:00, 2022-04-01 15:55:00)  306.670013  309.000000   
[2022-04-01 15:55:00, 2022-04-01 16:00:00)  308.630005  309.619995   

symbol                                                                       
                                                   low       close   volume  
[2022-03-31 09:35:00, 2022-03-31 09:40:00)  311.299988  312.005005   654932  
[2022-03-31 09:40:00, 2022-03-31 09:45:00)  311.350006  312.191895   372759  
[2022-03-31 09:45:00, 2022-03-31 09:50:00)  311.290009  311.940002   362373  
[2022-03-31 09:50:00, 2022-03-31 09:55:00)  311.519989  312.160004   317484  
[2022-03-31 09:55:00, 2022-03-31 10:00:00)  311.579987  311.885010   423869  
...                                                ...         ...      ...  
[2022-04-01 15:35:00, 2022-04-01 15:40:00)  306.209991  306.329987   241517  
[2022-04-01 15:40:00, 2022-04-01 15:45:00)  305.940002  306.114990   367177  
[2022-04-01 15:45:00, 2022-04-01 15:50:00)  305.851013  306.635010   396375  
[2022-04-01 15:50:00, 2022-04-01 15:55:00)  306.670013  308.665009  1101768  
[2022-04-01 15:55:00, 2022-04-01 16:00:00)  308.519989  309.420013  1636448  

[155 rows x 5 columns]

The previous first row is now excluded as including it would result in data being included for times earlier than `start`. The last row will be simlarly excluded if `end` is passed as one minute prior to the last session's close.

In [20]:
prices.get("5T", start + one_min, end - one_min)

symbol                                            MSFT              \
                                                  open        high   
[2022-03-31 09:35:00, 2022-03-31 09:40:00)  312.579987  312.625000   
[2022-03-31 09:40:00, 2022-03-31 09:45:00)  312.109985  312.515015   
[2022-03-31 09:45:00, 2022-03-31 09:50:00)  312.240387  312.500000   
[2022-03-31 09:50:00, 2022-03-31 09:55:00)  311.899994  312.179993   
[2022-03-31 09:55:00, 2022-03-31 10:00:00)  312.170013  312.309998   
...                                                ...         ...   
[2022-04-01 15:30:00, 2022-04-01 15:35:00)  307.000000  307.089996   
[2022-04-01 15:35:00, 2022-04-01 15:40:00)  306.429993  306.565002   
[2022-04-01 15:40:00, 2022-04-01 15:45:00)  306.299988  306.679993   
[2022-04-01 15:45:00, 2022-04-01 15:50:00)  306.100006  306.635010   
[2022-04-01 15:50:00, 2022-04-01 15:55:00)  306.670013  309.000000   

symbol                                                                       
                                                   low       close   volume  
[2022-03-31 09:35:00, 2022-03-31 09:40:00)  311.299988  312.005005   654932  
[2022-03-31 09:40:00, 2022-03-31 09:45:00)  311.350006  312.191895   372759  
[2022-03-31 09:45:00, 2022-03-31 09:50:00)  311.290009  311.940002   362373  
[2022-03-31 09:50:00, 2022-03-31 09:55:00)  311.519989  312.160004   317484  
[2022-03-31 09:55:00, 2022-03-31 10:00:00)  311.579987  311.885010   423869  
...                                                ...         ...      ...  
[2022-04-01 15:30:00, 2022-04-01 15:35:00)  306.309998  306.429993   236101  
[2022-04-01 15:35:00, 2022-04-01 15:40:00)  306.209991  306.329987   241517  
[2022-04-01 15:40:00, 2022-04-01 15:45:00)  305.940002  306.114990   367177  
[2022-04-01 15:45:00, 2022-04-01 15:50:00)  305.851013  306.635010   396375  
[2022-04-01 15:50:00, 2022-04-01 15:55:00)  306.670013  308.665009  1101768  

[154 rows x 5 columns]

1 minute data can also be used to illustrate how prices will not include any data that falls after `end`.

In [21]:
# set start and end 4 minutes apart and to times for which one minute data is available...
end = xnys.session_close(start_reg_T1).tz_convert(prices.tz_default) - one_min
start = end - pd.Timedelta(4, "T")
start, end

(Timestamp('2022-04-13 15:55:00-0400', tz='America/New_York'),
 Timestamp('2022-04-13 15:59:00-0400', tz='America/New_York'))

In [22]:
prices.get("1min", start, end)

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 15:55:00, 2022-04-13 15:56:00)  287.709991  287.806610   
[2022-04-13 15:56:00, 2022-04-13 15:57:00)  287.679993  287.825012   
[2022-04-13 15:57:00, 2022-04-13 15:58:00)  287.809998  287.829987   
[2022-04-13 15:58:00, 2022-04-13 15:59:00)  287.690002  287.700012   

symbol                                                                      
                                                   low       close  volume  
[2022-04-13 15:55:00, 2022-04-13 15:56:00)  287.570007  287.690002  117415  
[2022-04-13 15:56:00, 2022-04-13 15:57:00)  287.559998  287.804993   96092  
[2022-04-13 15:57:00, 2022-04-13 15:58:00)  287.640015  287.684998   77356  
[2022-04-13 15:58:00, 2022-04-13 15:59:00)  287.565002  287.595001  104963

The final indice runs to the 15:59 `end`. The minute from 15:59 to 16:00 is not included as it falls after `end`. The return will be the same if `end` is just one second short of 16:00.

In [23]:
end += pd.Timedelta(59, "S")
end

Timestamp('2022-04-13 15:59:59-0400', tz='America/New_York')

In [24]:
prices.get("1min", start, end)

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 15:55:00, 2022-04-13 15:56:00)  287.709991  287.806610   
[2022-04-13 15:56:00, 2022-04-13 15:57:00)  287.679993  287.825012   
[2022-04-13 15:57:00, 2022-04-13 15:58:00)  287.809998  287.829987   
[2022-04-13 15:58:00, 2022-04-13 15:59:00)  287.690002  287.700012   

symbol                                                                      
                                                   low       close  volume  
[2022-04-13 15:55:00, 2022-04-13 15:56:00)  287.570007  287.690002  117415  
[2022-04-13 15:56:00, 2022-04-13 15:57:00)  287.559998  287.804993   96092  
[2022-04-13 15:57:00, 2022-04-13 15:58:00)  287.640015  287.684998   77356  
[2022-04-13 15:58:00, 2022-04-13 15:59:00)  287.565002  287.595001  104963

Only when `end` covers the full minute leading to the session close will that last minute be included to the period.

In [25]:
end += pd.Timedelta(1, "S")
end

Timestamp('2022-04-13 16:00:00-0400', tz='America/New_York')

In [26]:
prices.get("1min", start, end)

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 15:55:00, 2022-04-13 15:56:00)  287.709991  287.806610   
[2022-04-13 15:56:00, 2022-04-13 15:57:00)  287.679993  287.825012   
[2022-04-13 15:57:00, 2022-04-13 15:58:00)  287.809998  287.829987   
[2022-04-13 15:58:00, 2022-04-13 15:59:00)  287.690002  287.700012   
[2022-04-13 15:59:00, 2022-04-13 16:00:00)  287.599915  287.839996   

symbol                                                                      
                                                   low       close  volume  
[2022-04-13 15:55:00, 2022-04-13 15:56:00)  287.570007  287.690002  117415  
[2022-04-13 15:56:00, 2022-04-13 15:57:00)  287.559998  287.804993   96092  
[2022-04-13 15:57:00, 2022-04-13 15:58:00)  287.640015  287.684998   77356  
[2022-04-13 15:58:00, 2022-04-13 15:59:00)  287.565002  287.595001  104963  
[2022-04-13 15:59:00, 2022-04-13 16:00:00)  287.299988  287.600006  363570

#### Timezones (`tzin`)

When `start` or `end` is passed as a time the timezone will be assumed as the default timezone of the symbols for which price data is being requested. The following is therefore interpreted as representing the local (New York) open and close of the respective sessions.

In [27]:
prices.get("1D", start="2021-12-01 09:30", end="2021-12-07 16:00")

symbol            MSFT                                              
                  open        high         low       close    volume
2021-12-01  335.130005  339.279999  329.390015  330.079987  33337600
2021-12-02  330.299988  333.489990  327.799988  329.489990  30766000
2021-12-03  331.989990  332.700012  318.029999  323.010010  41779300
2021-12-06  323.950012  327.450012  319.230011  326.190002  30032600
2021-12-07  331.640015  335.799988  330.100006  334.920013  31021900

The symbols' default timezone can be accessed by the prices' `tz_default` property.

In [28]:
prices.tz_default

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

If the input timezone is not intended to be interpreted as the default timezone then either:
* pass `start`/`end` as a timezone-aware `pd.Timestamp`.
* pass the intended timezone as `tzin`.

The following offers an example of passing timezone-aware `Timestamp` with timezone set as the Pacific side of the US.

In [29]:
# using the start and end values defined for the 1min interval example above
tz = pytz.timezone("US/Pacific")
start_pacific, end_pacific = start.astimezone(tz), end.astimezone(tz)
start_pacific, end_pacific

(Timestamp('2022-04-13 12:55:00-0700', tz='US/Pacific'),
 Timestamp('2022-04-13 13:00:00-0700', tz='US/Pacific'))

In [30]:
df = prices.get("1min", start_pacific, end_pacific)
df

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 15:55:00, 2022-04-13 15:56:00)  287.709991  287.806610   
[2022-04-13 15:56:00, 2022-04-13 15:57:00)  287.679993  287.825012   
[2022-04-13 15:57:00, 2022-04-13 15:58:00)  287.809998  287.829987   
[2022-04-13 15:58:00, 2022-04-13 15:59:00)  287.690002  287.700012   
[2022-04-13 15:59:00, 2022-04-13 16:00:00)  287.599915  287.839996   

symbol                                                                      
                                                   low       close  volume  
[2022-04-13 15:55:00, 2022-04-13 15:56:00)  287.570007  287.690002  117415  
[2022-04-13 15:56:00, 2022-04-13 15:57:00)  287.559998  287.804993   96092  
[2022-04-13 15:57:00, 2022-04-13 15:58:00)  287.640015  287.684998   77356  
[2022-04-13 15:58:00, 2022-04-13 15:59:00)  287.565002  287.595001  104963  
[2022-04-13 15:59:00, 2022-04-13 16:00:00)  287.299988  287.600006  363570

Notice that these inputs make no difference to the output timezone (which defaults to the `tz_default`, i.e New York).

In [31]:
df.pt.tz  # table's timezone

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

Or `start` and `end` can be passed without a timezone and `tzin` can be passed to define the timezone they represent... 

In [32]:
start_pacific = start_pacific.strftime("%Y-%m-%d %H:%M")
end_pacific = end_pacific.strftime("%Y-%m-%d %H:%M")
print(f"{start_pacific=}\n{end_pacific=}\n{tz=}")  # for reference

start_pacific='2022-04-13 12:55'
end_pacific='2022-04-13 13:00'
tz=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>


In [33]:
df = prices.get("1min", start_pacific, end_pacific, tzin=tz)
df

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 12:55:00, 2022-04-13 12:56:00)  287.709991  287.806610   
[2022-04-13 12:56:00, 2022-04-13 12:57:00)  287.679993  287.825012   
[2022-04-13 12:57:00, 2022-04-13 12:58:00)  287.809998  287.829987   
[2022-04-13 12:58:00, 2022-04-13 12:59:00)  287.690002  287.700012   
[2022-04-13 12:59:00, 2022-04-13 13:00:00)  287.599915  287.839996   

symbol                                                                      
                                                   low       close  volume  
[2022-04-13 12:55:00, 2022-04-13 12:56:00)  287.570007  287.690002  117415  
[2022-04-13 12:56:00, 2022-04-13 12:57:00)  287.559998  287.804993   96092  
[2022-04-13 12:57:00, 2022-04-13 12:58:00)  287.640015  287.684998   77356  
[2022-04-13 12:58:00, 2022-04-13 12:59:00)  287.565002  287.595001  104963  
[2022-04-13 12:59:00, 2022-04-13 13:00:00)  287.299988  287.600006  363570

Note that passing `tzin` also has the effect of changing the default output timezone to `tzin`...

In [34]:
df.pt.tz  # table's timezone

<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>

`tzout` can be passed to override the default behaviour.

In [35]:
tzout = pytz.timezone("US/Eastern")
df = prices.get("1min", start_pacific, end_pacific, tzin=tz, tzout=tzout)
df

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 15:55:00, 2022-04-13 15:56:00)  287.709991  287.806610   
[2022-04-13 15:56:00, 2022-04-13 15:57:00)  287.679993  287.825012   
[2022-04-13 15:57:00, 2022-04-13 15:58:00)  287.809998  287.829987   
[2022-04-13 15:58:00, 2022-04-13 15:59:00)  287.690002  287.700012   
[2022-04-13 15:59:00, 2022-04-13 16:00:00)  287.599915  287.839996   

symbol                                                                      
                                                   low       close  volume  
[2022-04-13 15:55:00, 2022-04-13 15:56:00)  287.570007  287.690002  117415  
[2022-04-13 15:56:00, 2022-04-13 15:57:00)  287.559998  287.804993   96092  
[2022-04-13 15:57:00, 2022-04-13 15:58:00)  287.640015  287.684998   77356  
[2022-04-13 15:58:00, 2022-04-13 15:59:00)  287.565002  287.595001  104963  
[2022-04-13 15:59:00, 2022-04-13 16:00:00)  287.299988  287.600006  363570

In [36]:
df.pt.tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

`tzin` can also be passed as a symbol, in which case the timezone of the input will be assumed as the symbol's timezone (an example is included to the later [tzin](#tzin) section).

### As dates and times
`start` and `end` are parsed independently, such that one can be defined as a date and the other as a time.

In [37]:
prices.get("1D", "2021-12-01 09:31", "2021-12-07")

symbol            MSFT                                              
                  open        high         low       close    volume
2021-12-02  330.299988  333.489990  327.799988  329.489990  30766000
2021-12-03  331.989990  332.700012  318.029999  323.010010  41779300
2021-12-06  323.950012  327.450012  319.230011  326.190002  30032600
2021-12-07  331.640015  335.799988  330.100006  334.920013  31021900

`start` and `end` will be treated as times if they have a time component that is not 00:00 and whenever they are passed as timezone-aware `pd.Timestamp`.

`start` and/or `end` can be defined as 'midnight' by passing as a timezone-aware `pd.Timestamp`. For example, compare the above with...

In [38]:
prices.get("1D", "2021-12-01 09:31", end=pd.Timestamp("2021-12-07", tz=pytz.UTC))

symbol            MSFT                                              
                  open        high         low       close    volume
2021-12-02  330.299988  333.489990  327.799988  329.489990  30766000
2021-12-03  331.989990  332.700012  318.029999  323.010010  41779300
2021-12-06  323.950012  327.450012  319.230011  326.190002  30032600

Notice that the 2021-12-07 session is no longer present. This is because the tz-aware `end` was interpreted as a time, and hence the period was evaluated through to 2021-12-07 00:00 UTC, i.e. way before the US market opened on 2021-12-07.

### Default values

If no period parameters are passed or only `start` is passed then `end` will default to 'now'.

If no period parameters are passed or only `end` is passed then `start` will default to the earliest date or time for which prices are available at the requested interval.

In [39]:
start = end = start_T1
start, end

(Timestamp('2022-04-13 00:00:00', freq='C'),
 Timestamp('2022-04-13 00:00:00', freq='C'))

In [40]:
prices.get("5T", start)

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 09:30:00, 2022-04-13 09:35:00)  282.049988  282.619995   
[2022-04-13 09:35:00, 2022-04-13 09:40:00)  281.769989  282.390015   
[2022-04-13 09:40:00, 2022-04-13 09:45:00)  281.769989  283.359985   
[2022-04-13 09:45:00, 2022-04-13 09:50:00)  283.119995  283.859985   
[2022-04-13 09:50:00, 2022-04-13 09:55:00)  283.230011  283.579193   
...                                                ...         ...   
[2022-05-12 15:35:00, 2022-05-12 15:40:00)  253.220001  253.899994   
[2022-05-12 15:40:00, 2022-05-12 15:45:00)  253.779999  254.410004   
[2022-05-12 15:45:00, 2022-05-12 15:50:00)  253.020004  253.699997   
[2022-05-12 15:50:00, 2022-05-12 15:55:00)  253.050003  254.660004   
[2022-05-12 15:55:00, 2022-05-12 16:00:00)  254.000000  255.779999   

symbol                                                                       
                                                   low       close   volume  
[2022-04-13 09:30:00, 2022-04-13 09:35:00)  281.600006  281.820007   961645  
[2022-04-13 09:35:00, 2022-04-13 09:40:00)  281.700012  281.750000   541153  
[2022-04-13 09:40:00, 2022-04-13 09:45:00)  281.299988  283.100006   767777  
[2022-04-13 09:45:00, 2022-04-13 09:50:00)  282.899994  283.250000   418059  
[2022-04-13 09:50:00, 2022-04-13 09:55:00)  282.920105  283.480011   286808  
...                                                ...         ...      ...  
[2022-05-12 15:35:00, 2022-05-12 15:40:00)  252.699997  253.750000   584798  
[2022-05-12 15:40:00, 2022-05-12 15:45:00)  252.589996  253.049896   713316  
[2022-05-12 15:45:00, 2022-05-12 15:50:00)  252.610001  252.880005   647195  
[2022-05-12 15:50:00, 2022-05-12 15:55:00)  252.970001  254.020004  1228553  
[2022-05-12 15:55:00, 2022-05-12 16:00:00)  253.820007  255.369995  2161222  

[1638 rows x 5 columns]

In [41]:
prices.get("5T", end=end)

symbol                                            MSFT              \
                                                  open        high   
[2022-03-14 09:30:00, 2022-03-14 09:35:00)  280.250000  282.359985   
[2022-03-14 09:35:00, 2022-03-14 09:40:00)  281.890015  282.850006   
[2022-03-14 09:40:00, 2022-03-14 09:45:00)  281.829987  284.769989   
[2022-03-14 09:45:00, 2022-03-14 09:50:00)  284.269989  284.329987   
[2022-03-14 09:50:00, 2022-03-14 09:55:00)  283.019989  283.699005   
...                                                ...         ...   
[2022-04-13 15:35:00, 2022-04-13 15:40:00)  288.450012  288.559906   
[2022-04-13 15:40:00, 2022-04-13 15:45:00)  288.410004  288.440002   
[2022-04-13 15:45:00, 2022-04-13 15:50:00)  288.140015  288.390015   
[2022-04-13 15:50:00, 2022-04-13 15:55:00)  288.040009  288.350006   
[2022-04-13 15:55:00, 2022-04-13 16:00:00)  287.709991  287.839996   

symbol                                                                         
                                                   low       close     volume  
[2022-03-14 09:30:00, 2022-03-14 09:35:00)  280.010010  281.869995  1721154.0  
[2022-03-14 09:35:00, 2022-03-14 09:40:00)  281.040009  281.880005   720010.0  
[2022-03-14 09:40:00, 2022-03-14 09:45:00)  281.369995  284.290009   808963.0  
[2022-03-14 09:45:00, 2022-03-14 09:50:00)  282.480011  283.109985   593177.0  
[2022-03-14 09:50:00, 2022-03-14 09:55:00)  282.470001  283.109985   367264.0  
...                                                ...         ...        ...  
[2022-04-13 15:35:00, 2022-04-13 15:40:00)  288.109985  288.420013   345646.0  
[2022-04-13 15:40:00, 2022-04-13 15:45:00)  288.059998  288.144989   351441.0  
[2022-04-13 15:45:00, 2022-04-13 15:50:00)  287.869995  288.059998   476969.0  
[2022-04-13 15:50:00, 2022-04-13 15:55:00)  287.470001  287.700012   574283.0  
[2022-04-13 15:55:00, 2022-04-13 16:00:00)  287.299988  287.600006   759396.0  

[1794 rows x 5 columns]

Note the earliest time for which 5 minute data is available...

In [42]:
prices.limits[prices.bis.T5][0].tz_convert(prices.tz_default)

Timestamp('2022-03-13 17:34:00-0400', tz='America/New_York')

If no parameters are passed then the interval defaults to daily and all available price history is returned.

In [43]:
prices.get()

symbol            MSFT                                                
                  open        high         low       close      volume
1986-03-13    0.088542    0.101563    0.088542    0.097222  1031788800
1986-03-14    0.097222    0.102431    0.097222    0.100694   308160000
1986-03-17    0.100694    0.103299    0.100694    0.102431   133171200
1986-03-18    0.102431    0.103299    0.098958    0.099826    67766400
1986-03-19    0.099826    0.100694    0.097222    0.098090    47894400
...                ...         ...         ...         ...         ...
2022-05-06  274.809998  279.250000  271.269989  274.730011    37748300
2022-05-09  270.059998  272.359985  263.320007  264.579987    47726000
2022-05-10  271.690002  273.750000  265.070007  269.500000    39336400
2022-05-11  265.679993  271.359985  259.299988  260.549988    48975900
2022-05-12  257.690002  259.880005  250.020004  255.350006    50875369

[9117 rows x 5 columns]

## Duration

The period can also be defined in terms of a duration, optionally bound by either `start` or `end`. If neither `start` nor `end` are passed then the duration will be bound on the right side by 'now'.

Durations can be defined in terms of either [trading time](#Trading-time) (`hours` and `minutes`), [trading sessions](#Trading-sessions) (`days`) or [calendar time](#Calendar-time) (`weeks`, `months` and `years`). 

### Trading time

The period parameters `minutes` and `hours` can be passed to define a duration in terms of 'trading time'. In this case the period will comprise the number of trading minutes described by `minutes` and/or `hours`.

If `start` is defined as a session (i.e. date) then the period will start on that session's open.

In [44]:
a_session = start_reg_T1
a_session

Timestamp('2022-04-13 00:00:00', freq='C')

In [45]:
prices.get("5T", a_session, minutes=20)  # first 20 minutes of a_session

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 09:30:00, 2022-04-13 09:35:00)  282.049988  282.619995   
[2022-04-13 09:35:00, 2022-04-13 09:40:00)  281.769989  282.390015   
[2022-04-13 09:40:00, 2022-04-13 09:45:00)  281.769989  283.359985   
[2022-04-13 09:45:00, 2022-04-13 09:50:00)  283.119995  283.859985   

symbol                                                                        
                                                   low       close    volume  
[2022-04-13 09:30:00, 2022-04-13 09:35:00)  281.600006  281.820007  961645.0  
[2022-04-13 09:35:00, 2022-04-13 09:40:00)  281.700012  281.750000  541153.0  
[2022-04-13 09:40:00, 2022-04-13 09:45:00)  281.299988  283.100006  767777.0  
[2022-04-13 09:45:00, 2022-04-13 09:50:00)  282.899994  283.250000  418059.0

Whilst if  `end` is defined as a session then the period will end on that session's close.

In [46]:
prices.get("15T", end=a_session, hours=1)  # last hour of a_session

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 15:00:00, 2022-04-13 15:15:00)  287.230011  287.916107   
[2022-04-13 15:15:00, 2022-04-13 15:30:00)  287.809998  288.160004   
[2022-04-13 15:30:00, 2022-04-13 15:45:00)  288.059998  288.579987   
[2022-04-13 15:45:00, 2022-04-13 16:00:00)  288.140015  288.390015   

symbol                                                                         
                                                   low       close     volume  
[2022-04-13 15:00:00, 2022-04-13 15:15:00)  286.769989  287.820007   684781.0  
[2022-04-13 15:15:00, 2022-04-13 15:30:00)  287.670013  288.065002   769447.0  
[2022-04-13 15:30:00, 2022-04-13 15:45:00)  287.910004  288.144989  1041082.0  
[2022-04-13 15:45:00, 2022-04-13 16:00:00)  287.299988  287.600006  1810648.0

To illustrate that we're dealing with trading time, rather than calendar time, consider what comes back when 48 hours of data is requested; not 2 days of prices, but rather around  7 sessions worth (48 / 6.5).

In [47]:
prices.get("30T", start=a_session, hours=48)

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 09:30:00, 2022-04-13 10:00:00)  282.049988  283.859985   
[2022-04-13 10:00:00, 2022-04-13 10:30:00)  282.779999  284.630005   
[2022-04-13 10:30:00, 2022-04-13 11:00:00)  284.100006  285.260010   
[2022-04-13 11:00:00, 2022-04-13 11:30:00)  283.640015  285.903198   
[2022-04-13 11:30:00, 2022-04-13 12:00:00)  285.570007  286.100006   
...                                                ...         ...   
[2022-04-25 09:30:00, 2022-04-25 10:00:00)  273.290009  275.179993   
[2022-04-25 10:00:00, 2022-04-25 10:30:00)  272.583710  277.279999   
[2022-04-25 10:30:00, 2022-04-25 11:00:00)  276.929993  278.019989   
[2022-04-25 11:00:00, 2022-04-25 11:30:00)  275.579987  276.049988   
[2022-04-25 11:30:00, 2022-04-25 12:00:00)  274.799988  275.799988   

symbol                                                                         
                                                   low       close     volume  
[2022-04-13 09:30:00, 2022-04-13 10:00:00)  281.299988  282.760010  3358871.0  
[2022-04-13 10:00:00, 2022-04-13 10:30:00)  282.358612  284.070007  1533721.0  
[2022-04-13 10:30:00, 2022-04-13 11:00:00)  283.510010  283.600006  1264834.0  
[2022-04-13 11:00:00, 2022-04-13 11:30:00)  283.614990  285.559998  1381601.0  
[2022-04-13 11:30:00, 2022-04-13 12:00:00)  285.100006  285.670013  1036898.0  
...                                                ...         ...        ...  
[2022-04-25 09:30:00, 2022-04-25 10:00:00)  270.769989  272.589996  5699576.0  
[2022-04-25 10:00:00, 2022-04-25 10:30:00)  272.470001  276.899994  3282787.0  
[2022-04-25 10:30:00, 2022-04-25 11:00:00)  275.410004  275.570190  2123040.0  
[2022-04-25 11:00:00, 2022-04-25 11:30:00)  273.799988  274.799988  1891263.0  
[2022-04-25 11:30:00, 2022-04-25 12:00:00)  272.820007  274.015015  1696405.0  

[96 rows x 5 columns]

The bound can also be defined as a time, for example 30 minutes after the open...

In [48]:
# Note: start is defined in terms of local tz only for ease of comparison with the
# output. It could be left as a UTC pd.Timestamp and the output would be the same.
a_time = xnys.session_open(a_session).tz_convert(prices.tz_default)
a_time += pd.Timedelta(30, "T")
a_time

Timestamp('2022-04-13 10:00:00-0400', tz='America/New_York')

In [49]:
prices.get("15T", a_time, hours=1, minutes=15)

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 10:00:00, 2022-04-13 10:15:00)  282.779999  283.850006   
[2022-04-13 10:15:00, 2022-04-13 10:30:00)  283.250000  284.630005   
[2022-04-13 10:30:00, 2022-04-13 10:45:00)  284.100006  285.260010   
[2022-04-13 10:45:00, 2022-04-13 11:00:00)  284.339996  284.755005   
[2022-04-13 11:00:00, 2022-04-13 11:15:00)  283.640015  285.420013   

symbol                                                                        
                                                   low       close    volume  
[2022-04-13 10:00:00, 2022-04-13 10:15:00)  282.358612  283.234985  765540.0  
[2022-04-13 10:15:00, 2022-04-13 10:30:00)  283.250000  284.070007  768181.0  
[2022-04-13 10:30:00, 2022-04-13 10:45:00)  283.510010  284.355011  704189.0  
[2022-04-13 10:45:00, 2022-04-13 11:00:00)  283.589996  283.600006  560645.0  
[2022-04-13 11:00:00, 2022-04-13 11:15:00)  283.614990  285.300110  758498.0

Consider what happens if this time is passed as `end`, thereby bounding the duration on the right.

In [50]:
prices.get("15T", end=a_time, hours=1, minutes=15)

symbol                                            MSFT              \
                                                  open        high   
[2022-04-12 15:15:00, 2022-04-12 15:30:00)  280.934998  281.820007   
[2022-04-12 15:30:00, 2022-04-12 15:45:00)  281.079987  282.070007   
[2022-04-12 15:45:00, 2022-04-12 16:00:00)  281.570007  282.579987   
[2022-04-13 09:30:00, 2022-04-13 09:45:00)  282.049988  283.359985   
[2022-04-13 09:45:00, 2022-04-13 10:00:00)  283.119995  283.859985   

symbol                                                                         
                                                   low       close     volume  
[2022-04-12 15:15:00, 2022-04-12 15:30:00)  280.709991  281.123993   960161.0  
[2022-04-12 15:30:00, 2022-04-12 15:45:00)  280.869995  281.589996  1198768.0  
[2022-04-12 15:45:00, 2022-04-12 16:00:00)  281.359985  281.980011  2169364.0  
[2022-04-13 09:30:00, 2022-04-13 09:45:00)  281.299988  283.100006  2270575.0  
[2022-04-13 09:45:00, 2022-04-13 10:00:00)  282.170013  282.760010  1088296.0

The period crosses sessions, ensuring that it represents one hour and 15 minutes of trading time ending `end`.

The above examples were nice and clean, not like the next one...

In [51]:
prices.get("25T", start=a_time, hours=1, minutes=40)

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 10:20:00, 2022-04-13 10:45:00)  284.019989  285.260010   
[2022-04-13 10:45:00, 2022-04-13 11:10:00)  284.339996  285.239990   
[2022-04-13 11:10:00, 2022-04-13 11:35:00)  284.609985  285.959991   
[2022-04-13 11:35:00, 2022-04-13 12:00:00)  285.670013  286.100006   

symbol                                                                         
                                                   low       close     volume  
[2022-04-13 10:20:00, 2022-04-13 10:45:00)  283.510010  284.355011  1218796.0  
[2022-04-13 10:45:00, 2022-04-13 11:10:00)  283.589996  284.619995  1078127.0  
[2022-04-13 11:10:00, 2022-04-13 11:35:00)  284.519989  285.649994  1085751.0  
[2022-04-13 11:35:00, 2022-04-13 12:00:00)  285.100006  285.670013   815266.0

The `start` time is 10.00, although the first indice starts at 10.20. Why the discrepancy? By default, price data is anchored on session opens*. Accordingly the initial indices of the session here would be 09.30 - 09.55, 09.55 - 10.20, 10.20 - 10.45... In accordance with the golden rule, prices will never include any data prior to `start`, so the first indice available is 10.20 - 10.45.

Further, note that the period ends at 12.00, which is 1 hour and 40 minutes (the duration) after 10.20, NOT 1 hour and 40 minutes after the 10.00 `start`. This demonstrates another rule for evaluating the period, the **Silver Rule** - **if the bound is unaligned then the other endpoint is evaluated from the next aligned period start/end, NOT from the passed `start`/`end`**.

(*NB see the [anchor](./anchor.ipynb) tutorial for how to anchor data on `end` and 'workback'.)

Consider what happens if the duration is extended by 24 mintues.

In [52]:
prices.get("25T", start=a_time, hours=2, minutes=4)

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 10:20:00, 2022-04-13 10:45:00)  284.019989  285.260010   
[2022-04-13 10:45:00, 2022-04-13 11:10:00)  284.339996  285.239990   
[2022-04-13 11:10:00, 2022-04-13 11:35:00)  284.609985  285.959991   
[2022-04-13 11:35:00, 2022-04-13 12:00:00)  285.670013  286.100006   

symbol                                                                         
                                                   low       close     volume  
[2022-04-13 10:20:00, 2022-04-13 10:45:00)  283.510010  284.355011  1218796.0  
[2022-04-13 10:45:00, 2022-04-13 11:10:00)  283.589996  284.619995  1078127.0  
[2022-04-13 11:10:00, 2022-04-13 11:35:00)  284.519989  285.649994  1085751.0  
[2022-04-13 11:35:00, 2022-04-13 12:00:00)  285.100006  285.670013   815266.0

The return hasn't changed because including an extra indice, spanning 25 minutes, would result in the return covering a period longer than the requested duration. In accordance with the **Golden Rule**, the return never includes prices for any time period in excess of what's requested.

Adding a further minute includes the next indice...

In [53]:
prices.get("25T", start=a_time, hours=2, minutes=5)

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 10:20:00, 2022-04-13 10:45:00)  284.019989  285.260010   
[2022-04-13 10:45:00, 2022-04-13 11:10:00)  284.339996  285.239990   
[2022-04-13 11:10:00, 2022-04-13 11:35:00)  284.609985  285.959991   
[2022-04-13 11:35:00, 2022-04-13 12:00:00)  285.670013  286.100006   
[2022-04-13 12:00:00, 2022-04-13 12:25:00)  285.660004  286.339996   

symbol                                                                         
                                                   low       close     volume  
[2022-04-13 10:20:00, 2022-04-13 10:45:00)  283.510010  284.355011  1218796.0  
[2022-04-13 10:45:00, 2022-04-13 11:10:00)  283.589996  284.619995  1078127.0  
[2022-04-13 11:10:00, 2022-04-13 11:35:00)  284.519989  285.649994  1085751.0  
[2022-04-13 11:35:00, 2022-04-13 12:00:00)  285.100006  285.670013   815266.0  
[2022-04-13 12:00:00, 2022-04-13 12:25:00)  285.230011  286.174988   818918.0

If no bound is passed then `end` defaults to 'now'. So, to get the last 20 mintues of trading data...

In [54]:
prices.get("5T", minutes=20)

symbol                                            MSFT              \
                                                  open        high   
[2022-05-12 15:40:00, 2022-05-12 15:45:00)  253.779999  254.410004   
[2022-05-12 15:45:00, 2022-05-12 15:50:00)  253.020004  253.699997   
[2022-05-12 15:50:00, 2022-05-12 15:55:00)  253.050003  254.660004   
[2022-05-12 15:55:00, 2022-05-12 16:00:00)  254.000000  255.779999   

symbol                                                                         
                                                   low       close     volume  
[2022-05-12 15:40:00, 2022-05-12 15:45:00)  252.589996  253.049896   713316.0  
[2022-05-12 15:45:00, 2022-05-12 15:50:00)  252.610001  252.880005   647195.0  
[2022-05-12 15:50:00, 2022-05-12 15:55:00)  252.970001  254.020004  1228553.0  
[2022-05-12 15:55:00, 2022-05-12 16:00:00)  253.820007  255.369995  2161222.0

If the symbol is currently trading then the last interval will be a 'live interval' which encompasses 'now' (or 'now' - 'delay' in the event prices are delayed). Live intervals contribute to the duration in full.

### Trading sessions

The period parameter `days` can be passed to define a duration as a specific number of 'trading sessions'.

In [55]:
print(f"{a_session=}\n")  # for reference
prices.get("5T", start=a_session, days=7) # 7 sessions of data from a_session

a_session=Timestamp('2022-04-13 00:00:00', freq='C')



symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 09:30:00, 2022-04-13 09:35:00)  282.049988  282.619995   
[2022-04-13 09:35:00, 2022-04-13 09:40:00)  281.769989  282.390015   
[2022-04-13 09:40:00, 2022-04-13 09:45:00)  281.769989  283.359985   
[2022-04-13 09:45:00, 2022-04-13 09:50:00)  283.119995  283.859985   
[2022-04-13 09:50:00, 2022-04-13 09:55:00)  283.230011  283.579193   
...                                                ...         ...   
[2022-04-22 15:35:00, 2022-04-22 15:40:00)  277.369995  277.660004   
[2022-04-22 15:40:00, 2022-04-22 15:45:00)  276.750000  277.250000   
[2022-04-22 15:45:00, 2022-04-22 15:50:00)  276.260010  276.290009   
[2022-04-22 15:50:00, 2022-04-22 15:55:00)  275.200012  275.279999   
[2022-04-22 15:55:00, 2022-04-22 16:00:00)  274.399994  274.619995   

symbol                                                                         
                                                   low       close     volume  
[2022-04-13 09:30:00, 2022-04-13 09:35:00)  281.600006  281.820007   961645.0  
[2022-04-13 09:35:00, 2022-04-13 09:40:00)  281.700012  281.750000   541153.0  
[2022-04-13 09:40:00, 2022-04-13 09:45:00)  281.299988  283.100006   767777.0  
[2022-04-13 09:45:00, 2022-04-13 09:50:00)  282.899994  283.250000   418059.0  
[2022-04-13 09:50:00, 2022-04-13 09:55:00)  282.920105  283.480011   286808.0  
...                                                ...         ...        ...  
[2022-04-22 15:35:00, 2022-04-22 15:40:00)  276.619995  276.690002   356469.0  
[2022-04-22 15:40:00, 2022-04-22 15:45:00)  275.940002  276.225006   457243.0  
[2022-04-22 15:45:00, 2022-04-22 15:50:00)  275.140015  275.170013   431023.0  
[2022-04-22 15:50:00, 2022-04-22 15:55:00)  273.940002  274.390015  1607916.0  
[2022-04-22 15:55:00, 2022-04-22 16:00:00)  273.380005  274.029999  2041577.0  

[546 rows x 5 columns]

The period can be seen to comprise 7 sessions of data, not 7 calendar days of data.

To get the last 10 sessions of daily data...

In [56]:
prices.get("1D", days=10)

symbol            MSFT                                              
                  open        high         low       close    volume
2022-04-29  288.609985  289.880005  276.500000  277.519989  37025000
2022-05-02  277.709991  284.940002  276.220001  284.470001  35151100
2022-05-03  283.959991  284.130005  280.149994  281.779999  25978600
2022-05-04  282.589996  290.880005  276.730011  289.980011  33599300
2022-05-05  285.540009  286.350006  274.339996  277.350006  43260400
2022-05-06  274.809998  279.250000  271.269989  274.730011  37748300
2022-05-09  270.059998  272.359985  263.320007  264.579987  47726000
2022-05-10  271.690002  273.750000  265.070007  269.500000  39336400
2022-05-11  265.679993  271.359985  259.299988  260.549988  48975900
2022-05-12  257.690002  259.880005  250.020004  255.350006  50875369

If the bound is passed as a time then the period will be evaluated to the same time of the session at the other end of the period.

In [57]:
start = xnys.session_open(start_reg_T1).astimezone(prices.tz_default)
start += pd.Timedelta(3, "H")
start

Timestamp('2022-04-13 12:30:00-0400', tz='America/New_York')

In [58]:
prices.get("10T", start=start, days=2)

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 12:30:00, 2022-04-13 12:40:00)  286.089996  286.318512   
[2022-04-13 12:40:00, 2022-04-13 12:50:00)  286.250000  286.619995   
[2022-04-13 12:50:00, 2022-04-13 13:00:00)  286.299988  286.415009   
[2022-04-13 13:00:00, 2022-04-13 13:10:00)  286.100006  286.440002   
[2022-04-13 13:10:00, 2022-04-13 13:20:00)  286.269989  286.739990   
...                                                ...         ...   
[2022-04-18 11:40:00, 2022-04-18 11:50:00)  279.760010  280.070007   
[2022-04-18 11:50:00, 2022-04-18 12:00:00)  279.420013  280.019989   
[2022-04-18 12:00:00, 2022-04-18 12:10:00)  279.959991  280.070007   
[2022-04-18 12:10:00, 2022-04-18 12:20:00)  279.859985  279.940002   
[2022-04-18 12:20:00, 2022-04-18 12:30:00)  279.280090  279.410004   

symbol                                                                        
                                                   low       close    volume  
[2022-04-13 12:30:00, 2022-04-13 12:40:00)  285.829987  286.220001  323074.0  
[2022-04-13 12:40:00, 2022-04-13 12:50:00)  286.095001  286.300110  282169.0  
[2022-04-13 12:50:00, 2022-04-13 13:00:00)  286.023407  286.084991  275360.0  
[2022-04-13 13:00:00, 2022-04-13 13:10:00)  285.970001  286.279999  355638.0  
[2022-04-13 13:10:00, 2022-04-13 13:20:00)  286.225006  286.429993  267721.0  
...                                                ...         ...       ...  
[2022-04-18 11:40:00, 2022-04-18 11:50:00)  279.399994  279.399994  297978.0  
[2022-04-18 11:50:00, 2022-04-18 12:00:00)  279.359985  279.959991  252773.0  
[2022-04-18 12:00:00, 2022-04-18 12:10:00)  279.700012  279.859985  334546.0  
[2022-04-18 12:10:00, 2022-04-18 12:20:00)  279.170013  279.290009  447852.0  
[2022-04-18 12:20:00, 2022-04-18 12:30:00)  278.750000  278.799988  371849.0  

[78 rows x 5 columns]

### Calendar time

Lastly, the period parameters `weeks`, `months` and `years` can be passed to define a duration in terms of calendar time - what the parameters say on the tin.

In [59]:
prices.get(start="2021-01-01", weeks=1)

symbol            MSFT                                              
                  open        high         low       close    volume
2021-01-04  222.529999  223.000000  214.809998  217.690002  37130100
2021-01-05  217.259995  218.520004  215.699997  217.899994  23823000
2021-01-06  212.169998  216.490005  211.940002  212.250000  35930700
2021-01-07  214.039993  219.339996  213.710007  218.289993  27694500
2021-01-08  218.679993  220.580002  217.029999  219.619995  22956200

Note that in line with the **Silver Rule**, the week is evaluated NOT from `start` but rather from the start of the evaluated period (in this case 2021-01-04, the first session of 2021).

Other examples...

In [60]:
prices.get("6D", start="2021-01-01", months=3)

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-01-06, 2021-01-14)  212.169998  220.580002  211.940002  216.339996   
[2021-01-14, 2021-01-25)  215.910004  230.070007  212.029999  225.949997   
[2021-01-25, 2021-02-02)  229.119995  242.639999  224.220001  239.649994   
[2021-02-02, 2021-02-10)  241.300003  245.089996  238.690002  243.770004   
[2021-02-10, 2021-02-19)  245.000000  246.130005  240.860001  243.789993   
[2021-02-19, 2021-03-01)  243.750000  243.860001  227.880005  232.380005   
[2021-03-01, 2021-03-09)  235.899994  237.470001  224.259995  227.389999   
[2021-03-09, 2021-03-17)  232.880005  240.059998  231.669998  237.710007   
[2021-03-17, 2021-03-25)  236.149994  241.050003  229.350006  235.460007   
[2021-03-25, 2021-04-02)  235.300003  242.839996  231.100006  242.350006   

symbol                                 
                               volume  
[2021-01-06, 2021-01-14)  152949100.0  
[2021-01-14, 2021-01-25)  190367600.0  
[2021-01-25, 2021-02-02)  277120800.0  
[2021-02-02, 2021-02-10)  142202200.0  
[2021-02-10, 2021-02-19)  119806500.0  
[2021-02-19, 2021-03-01)  195639300.0  
[2021-03-01, 2021-03-09)  204034000.0  
[2021-03-09, 2021-03-17)  169515700.0  
[2021-03-17, 2021-03-25)  198211300.0  
[2021-03-25, 2021-04-02)  183522800.0

In [61]:
prices.get("2M", start="2020", years=2, months=2)

symbol                          MSFT                                      \
                                open        high         low       close   
[2020-01-01, 2020-03-01)  158.779999  190.699997  152.000000  162.009995   
[2020-03-01, 2020-05-01)  165.309998  180.399994  132.520004  179.210007   
[2020-05-01, 2020-07-01)  175.800003  204.399994  173.800003  203.509995   
[2020-07-01, 2020-09-01)  203.139999  231.149994  197.509995  225.529999   
[2020-09-01, 2020-11-01)  225.509995  232.860001  196.250000  202.470001   
[2020-11-01, 2021-01-01)  204.289993  228.119995  200.119995  222.419998   
[2021-01-01, 2021-03-01)  222.529999  246.130005  211.940002  232.380005   
[2021-03-01, 2021-05-01)  235.899994  263.190002  224.259995  252.179993   
[2021-05-01, 2021-07-01)  253.399994  271.649994  238.070007  270.899994   
[2021-07-01, 2021-09-01)  269.609985  305.839996  269.600006  301.880005   
[2021-09-01, 2021-11-01)  302.869995  332.000000  280.250000  331.619995   
[2021-11-01, 2022-01-01)  331.359985  349.670013  317.250000  336.320007   
[2022-01-01, 2022-03-01)  335.350006  338.000000  271.519989  298.790009   

symbol                                  
                                volume  
[2020-01-01, 2020-03-01)  1.446052e+09  
[2020-03-01, 2020-05-01)  2.597400e+09  
[2020-05-01, 2020-07-01)  1.453810e+09  
[2020-07-01, 2020-09-01)  1.462615e+09  
[2020-09-01, 2020-11-01)  1.399794e+09  
[2020-11-01, 2021-01-01)  1.168205e+09  
[2021-01-01, 2021-03-01)  1.139039e+09  
[2021-03-01, 2021-05-01)  1.293607e+09  
[2021-05-01, 2021-07-01)  1.003657e+09  
[2021-07-01, 2021-09-01)  9.639816e+08  
[2021-09-01, 2021-11-01)  1.019434e+09  
[2021-11-01, 2022-01-01)  1.135560e+09  
[2022-01-01, 2022-03-01)  1.644582e+09

## Invalid period parameters combinations

It is not posssible to combine period parameters that describe durations in different terms. For example, whilst `weeks` can be combined with `months` or `years`, it can't be passed together with `days`.

In [ ]:
prices.get("1D", "2021", weeks=5, days=2)

```---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-112-4cf5b79cd115> in <module>
----> 1 prices.get("1D", "2021", weeks=5, days=2)

ValueError: `days` cannot be combined with other duration components.
```

And similarly `hours` can be combined with `minutes`, but not `weeks`.

In [ ]:
prices.get("1D", "2021-12-01", hours=5, weeks=2)

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-93-f444ad145a46> in <module>
----> 1 prices.get("1D", "2021-12-01", hours=5, weeks=2)

ValueError: `hours` and `minutes` cannot be combined with other duration components.
```

And passing both bounds and a duration ain't gonna fly...

In [ ]:
prices.get("1D", "2021-12-01", "2021-12-31", days=20)

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-117-6f433330ec2c> in <module>
----> 1 prices.get("1D", "2021-12-01", "2021-12-31", days=20)

ValueError: If pass start and end then cannot pass a duration component.
```

## Multiple exchanges (`lead_symbol`)

Run the following cell to instantiate a price object for this part of this tutorial.

In [65]:
# prices for Microsoft and Bitcoin
prices = PricesYahoo("MSFT, BTC-USD")

To evaluate periods `market_prices` refers to a calendar that has knowledge of the days and times that the exchange is open.

(To learn more about calendars see the [tutorials](https://github.com/gerrymanoim/exchange_calendars#tutorials) of the [`exchange_calendars`](https://github.com/gerrymanoim/exchange_calendars) library.)

But what happens when you request prices for multiple symbols that trade on different exchanges with differing opening hours and days?

The prices object instantiated in the cell above has two symbols, Microsoft and Bitcoin. MSFT trades Mon-Fri on the NYSE, usually from 09.30 through 16.00 local time. In contrast Bitcoin trades 24/7. So, if the last three sessions of data are requested ending on a Monday, should prices be returned for the last three Bitcoin sessions, which includes sessions over the weekend, or the last three sessions according to the calendar that MSFT trades on, which would cover more than three Bitcoin sessions?

To resolve this ambiguity the period is evaluated against the calendar associated with a 'lead symbol'. The `get` method takes an optional `lead_symbol` argument to define a symbol against which to evaluate the period.

Exploring the example of asking for three sessions of prices ending on a Monday...

In [66]:
prices.get("D", end="2022-01-24", days=3, lead_symbol="BTC-USD")

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2022-01-22         NaN         NaN         NaN         NaN         NaN   
2022-01-23         NaN         NaN         NaN         NaN         NaN   
2022-01-24  292.200012  297.109985  276.049988  296.369995  85731500.0   

symbol           BTC-USD                                            \
                    open          high           low         close   
2022-01-22  36471.589844  36688.812500  34349.250000  35030.250000   
2022-01-23  35047.359375  36433.312500  34784.968750  36276.804688   
2022-01-24  36275.734375  37247.519531  33184.058594  36654.328125   

symbol                   
                 volume  
2022-01-22  39714385405  
2022-01-23  26017975951  
2022-01-24  41856658597

As expected, when Bitcoin is the lead symbol the return covers three sessions. Note that prices are not available for MSFT over the weekend.

Changing `lead_symbol` to MSFT...

In [67]:
prices.get("D", end="2022-01-24", days=3, lead_symbol="MSFT")

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2022-01-20  309.070007  311.649994  301.140015  301.600006  35380700.0   
2022-01-21  302.690002  304.109985  295.609985  296.029999  57984400.0   
2022-01-22         NaN         NaN         NaN         NaN         NaN   
2022-01-23         NaN         NaN         NaN         NaN         NaN   
2022-01-24  292.200012  297.109985  276.049988  296.369995  85731500.0   

symbol           BTC-USD                                            \
                    open          high           low         close   
2022-01-20  41744.027344  43413.023438  40672.824219  40680.417969   
2022-01-21  40699.605469  41060.527344  35791.425781  36457.316406   
2022-01-22  36471.589844  36688.812500  34349.250000  35030.250000   
2022-01-23  35047.359375  36433.312500  34784.968750  36276.804688   
2022-01-24  36275.734375  37247.519531  33184.058594  36654.328125   

symbol                   
                 volume  
2022-01-20  20382033940  
2022-01-21  43011992031  
2022-01-22  39714385405  
2022-01-23  26017975951  
2022-01-24  41856658597

The period is now evaluated to cover the last three MSFT sessions, which consequently covers five Bitcoin sessions.

The same principle can be seen with intraday data by requesting prices from a `start` one hour before a MSFT open.

In [68]:
start = xnys.session_open(start_reg_T1).astimezone(prices.tz_default)
start -= pd.Timedelta(1, "H")
start

Timestamp('2022-04-13 08:30:00-0400', tz='America/New_York')

In [69]:
prices.get("6T", start, minutes=30, lead_symbol="MSFT")

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 09:30:00, 2022-04-13 09:36:00)  282.049988  282.619995   
[2022-04-13 09:36:00, 2022-04-13 09:42:00)  282.079987  282.369995   
[2022-04-13 09:42:00, 2022-04-13 09:48:00)  282.140015  283.859985   
[2022-04-13 09:48:00, 2022-04-13 09:54:00)  283.660004  283.779999   
[2022-04-13 09:54:00, 2022-04-13 10:00:00)  283.059998  283.540009   

symbol                                                                         \
                                                   low       close     volume   
[2022-04-13 09:30:00, 2022-04-13 09:36:00)  281.600006  282.000000  1109771.0   
[2022-04-13 09:36:00, 2022-04-13 09:42:00)  281.299988  282.174988   731246.0   
[2022-04-13 09:42:00, 2022-04-13 09:48:00)  282.000000  283.635010   682904.0   
[2022-04-13 09:48:00, 2022-04-13 09:54:00)  282.920105  283.079315   403019.0   
[2022-04-13 09:54:00, 2022-04-13 10:00:00)  282.170013  282.760010   431931.0   

symbol                                           BTC-USD                \
                                                    open          high   
[2022-04-13 09:30:00, 2022-04-13 09:36:00)  39844.710938  39884.253906   
[2022-04-13 09:36:00, 2022-04-13 09:42:00)  39900.550781  40081.367188   
[2022-04-13 09:42:00, 2022-04-13 09:48:00)  40077.402344  40272.574219   
[2022-04-13 09:48:00, 2022-04-13 09:54:00)  40296.570312  40392.902344   
[2022-04-13 09:54:00, 2022-04-13 10:00:00)  40410.648438  40517.910156   

symbol                                                                  \
                                                     low         close   
[2022-04-13 09:30:00, 2022-04-13 09:36:00)  39844.710938  39884.253906   
[2022-04-13 09:36:00, 2022-04-13 09:42:00)  39900.550781  40075.390625   
[2022-04-13 09:42:00, 2022-04-13 09:48:00)  40077.402344  40272.574219   
[2022-04-13 09:48:00, 2022-04-13 09:54:00)  40296.570312  40392.902344   
[2022-04-13 09:54:00, 2022-04-13 10:00:00)  40410.648438  40517.910156   

symbol                                                    
                                                  volume  
[2022-04-13 09:30:00, 2022-04-13 09:36:00)  9.173197e+07  
[2022-04-13 09:36:00, 2022-04-13 09:42:00)  4.013343e+10  
[2022-04-13 09:42:00, 2022-04-13 09:48:00)  1.619476e+08  
[2022-04-13 09:48:00, 2022-04-13 09:54:00)  1.714565e+08  
[2022-04-13 09:54:00, 2022-04-13 10:00:00)  4.072979e+10

Although `start` was passed with time 08:30, as the lead symbol is MSFT the evaluated period doesn't actually start until the NYSE open at 09:30. However if the lead symbol is the 24/7 trading Bitcoin then the evaluated period covers the 30 minutes from `start` through 09:00, i.e. before the NYSE opens...

In [70]:
prices.get(
    "6T", start, minutes=30, lead_symbol="BTC-USD", tzout=prices.tz_default
)

symbol                                     MSFT                        \
                                           open high low close volume   
[2022-04-13 08:30:00, 2022-04-13 08:36:00)  NaN  NaN NaN   NaN    NaN   
[2022-04-13 08:36:00, 2022-04-13 08:42:00)  NaN  NaN NaN   NaN    NaN   
[2022-04-13 08:42:00, 2022-04-13 08:48:00)  NaN  NaN NaN   NaN    NaN   
[2022-04-13 08:48:00, 2022-04-13 08:54:00)  NaN  NaN NaN   NaN    NaN   
[2022-04-13 08:54:00, 2022-04-13 09:00:00)  NaN  NaN NaN   NaN    NaN   

symbol                                           BTC-USD                \
                                                    open          high   
[2022-04-13 08:30:00, 2022-04-13 08:36:00)  39772.921875  39798.882812   
[2022-04-13 08:36:00, 2022-04-13 08:42:00)  39785.152344  39785.152344   
[2022-04-13 08:42:00, 2022-04-13 08:48:00)  39771.691406  39834.062500   
[2022-04-13 08:48:00, 2022-04-13 08:54:00)  39828.746094  39828.746094   
[2022-04-13 08:54:00, 2022-04-13 09:00:00)  39803.710938  39803.710938   

symbol                                                                  \
                                                     low         close   
[2022-04-13 08:30:00, 2022-04-13 08:36:00)  39772.921875  39793.957031   
[2022-04-13 08:36:00, 2022-04-13 08:42:00)  39747.753906  39754.417969   
[2022-04-13 08:42:00, 2022-04-13 08:48:00)  39771.691406  39834.062500   
[2022-04-13 08:48:00, 2022-04-13 08:54:00)  39816.457031  39816.457031   
[2022-04-13 08:54:00, 2022-04-13 09:00:00)  39757.296875  39757.296875   

symbol                                                
                                              volume  
[2022-04-13 08:30:00, 2022-04-13 08:36:00)       0.0  
[2022-04-13 08:36:00, 2022-04-13 08:42:00)       0.0  
[2022-04-13 08:42:00, 2022-04-13 08:48:00)  751616.0  
[2022-04-13 08:48:00, 2022-04-13 08:54:00)       0.0  
[2022-04-13 08:54:00, 2022-04-13 09:00:00)       0.0

Note: `tzout` was passed above for the purposes of comparison, by default prices are returned in the timezone associated with lead symbol (UTC in the case of Bitcoin)...

In [71]:
df = prices.get("6T", start, minutes=30, lead_symbol="BTC-USD")
df

symbol                                     MSFT                        \
                                           open high low close volume   
[2022-04-13 12:30:00, 2022-04-13 12:36:00)  NaN  NaN NaN   NaN    NaN   
[2022-04-13 12:36:00, 2022-04-13 12:42:00)  NaN  NaN NaN   NaN    NaN   
[2022-04-13 12:42:00, 2022-04-13 12:48:00)  NaN  NaN NaN   NaN    NaN   
[2022-04-13 12:48:00, 2022-04-13 12:54:00)  NaN  NaN NaN   NaN    NaN   
[2022-04-13 12:54:00, 2022-04-13 13:00:00)  NaN  NaN NaN   NaN    NaN   

symbol                                           BTC-USD                \
                                                    open          high   
[2022-04-13 12:30:00, 2022-04-13 12:36:00)  39772.921875  39798.882812   
[2022-04-13 12:36:00, 2022-04-13 12:42:00)  39785.152344  39785.152344   
[2022-04-13 12:42:00, 2022-04-13 12:48:00)  39771.691406  39834.062500   
[2022-04-13 12:48:00, 2022-04-13 12:54:00)  39828.746094  39828.746094   
[2022-04-13 12:54:00, 2022-04-13 13:00:00)  39803.710938  39803.710938   

symbol                                                                  \
                                                     low         close   
[2022-04-13 12:30:00, 2022-04-13 12:36:00)  39772.921875  39793.957031   
[2022-04-13 12:36:00, 2022-04-13 12:42:00)  39747.753906  39754.417969   
[2022-04-13 12:42:00, 2022-04-13 12:48:00)  39771.691406  39834.062500   
[2022-04-13 12:48:00, 2022-04-13 12:54:00)  39816.457031  39816.457031   
[2022-04-13 12:54:00, 2022-04-13 13:00:00)  39757.296875  39757.296875   

symbol                                                
                                              volume  
[2022-04-13 12:30:00, 2022-04-13 12:36:00)       0.0  
[2022-04-13 12:36:00, 2022-04-13 12:42:00)       0.0  
[2022-04-13 12:42:00, 2022-04-13 12:48:00)  751616.0  
[2022-04-13 12:48:00, 2022-04-13 12:54:00)       0.0  
[2022-04-13 12:54:00, 2022-04-13 13:00:00)       0.0

In [72]:
df.pt.tz

<UTC>

### Default calendar
If `lead_symbol` is not passed the period will be evaluated according to the default calendar. This in turn defaults to the most common calendar of all symbols or, in the event of a tie, the first of the tied symbols received to the Prices class as `symbols`. (See the [prices](./prices.ipynb) tutorial for an explanation of how calendars are associated with symbols.)

The default calendar can be queried via the prices object's `calendar_default` property.

In [73]:
prices.calendar_default, prices.calendar_default.name

(<exchange_calendars.exchange_calendar_xnys.XNYSExchangeCalendar at 0x20ac50df400>,
 'XNYS')

So here if `lead_symbol` is not passed to `get` then the period will be evaluated according to the NYSE calendar, as can be seen by repeating the three sessions example without passing `lead_symbol`...

In [74]:
prices.get("D", end="2022-01-24", days=3)

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2022-01-20  309.070007  311.649994  301.140015  301.600006  35380700.0   
2022-01-21  302.690002  304.109985  295.609985  296.029999  57984400.0   
2022-01-22         NaN         NaN         NaN         NaN         NaN   
2022-01-23         NaN         NaN         NaN         NaN         NaN   
2022-01-24  292.200012  297.109985  276.049988  296.369995  85731500.0   

symbol           BTC-USD                                            \
                    open          high           low         close   
2022-01-20  41744.027344  43413.023438  40672.824219  40680.417969   
2022-01-21  40699.605469  41060.527344  35791.425781  36457.316406   
2022-01-22  36471.589844  36688.812500  34349.250000  35030.250000   
2022-01-23  35047.359375  36433.312500  34784.968750  36276.804688   
2022-01-24  36275.734375  37247.519531  33184.058594  36654.328125   

symbol                   
                 volume  
2022-01-20  20382033940  
2022-01-21  43011992031  
2022-01-22  39714385405  
2022-01-23  26017975951  
2022-01-24  41856658597

The default calendar can be overriden for all calls to `get` by passing `lead_symbol` to the Prices class when instantiating the prices object. So to set the default calendar to the calendar associated with Bitcoin...  

In [75]:
prices = PricesYahoo("MSFT, BTC-USD", lead_symbol="BTC-USD")

In [76]:
prices.calendar_default, prices.calendar_default.name

(<exchange_calendars.always_open.AlwaysOpenCalendar at 0x20acb523790>, '24/7')

Such that the same call to `get` will now evaluate the period according to the calendar associated with Bitcoin...

In [77]:
prices.get("D", end="2022-01-24", days=3)

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2022-01-22         NaN         NaN         NaN         NaN         NaN   
2022-01-23         NaN         NaN         NaN         NaN         NaN   
2022-01-24  292.200012  297.109985  276.049988  296.369995  85731500.0   

symbol           BTC-USD                                            \
                    open          high           low         close   
2022-01-22  36471.589844  36688.812500  34349.250000  35030.250000   
2022-01-23  35047.359375  36433.312500  34784.968750  36276.804688   
2022-01-24  36275.734375  37247.519531  33184.058594  36654.328125   

symbol                   
                 volume  
2022-01-22  39714385405  
2022-01-23  26017975951  
2022-01-24  41856658597

And the lead symbol can always be overriden for any specific call to `get`...

In [78]:
prices.get("D", end="2022-01-24", days=3, lead_symbol="MSFT")

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2022-01-20  309.070007  311.649994  301.140015  301.600006  35380700.0   
2022-01-21  302.690002  304.109985  295.609985  296.029999  57984400.0   
2022-01-22         NaN         NaN         NaN         NaN         NaN   
2022-01-23         NaN         NaN         NaN         NaN         NaN   
2022-01-24  292.200012  297.109985  276.049988  296.369995  85731500.0   

symbol           BTC-USD                                            \
                    open          high           low         close   
2022-01-20  41744.027344  43413.023438  40672.824219  40680.417969   
2022-01-21  40699.605469  41060.527344  35791.425781  36457.316406   
2022-01-22  36471.589844  36688.812500  34349.250000  35030.250000   
2022-01-23  35047.359375  36433.312500  34784.968750  36276.804688   
2022-01-24  36275.734375  37247.519531  33184.058594  36654.328125   

symbol                   
                 volume  
2022-01-20  20382033940  
2022-01-21  43011992031  
2022-01-22  39714385405  
2022-01-23  26017975951  
2022-01-24  41856658597

### `tzin`

As touched on in the earlier [Timezones (`tzin`)](#Timezones-(tzin)) subsection, a symbol can be passed to `tzin` to define the timezone that should be attributed to `start` and `end` (when those arguments are NOT otherwise passed as timezone-aware `pd.Timestamp`).

In [79]:
# get `start` as a string that reflects an hour after an NYSE open.
start = xnys.session_open(start_reg_T1).astimezone(xnys.tz)
start += pd.Timedelta(1, "H")
start = start.strftime("%Y-%m-%d %H:%M")
start

'2022-04-13 10:30'

Compare the following where the time is considered to have the same timezone as the NYSE on which MSFT trades...

In [80]:
prices.get("15T", start, hours=1, tzin="MSFT", tzout="MSFT")

symbol                                            MSFT              \
                                                  open        high   
[2022-04-13 10:30:00, 2022-04-13 10:45:00)  284.100006  285.260010   
[2022-04-13 10:45:00, 2022-04-13 11:00:00)  284.339996  284.755005   
[2022-04-13 11:00:00, 2022-04-13 11:15:00)  283.640015  285.420013   
[2022-04-13 11:15:00, 2022-04-13 11:30:00)  285.290009  285.903198   

symbol                                                                        \
                                                   low       close    volume   
[2022-04-13 10:30:00, 2022-04-13 10:45:00)  283.510010  284.355011  704189.0   
[2022-04-13 10:45:00, 2022-04-13 11:00:00)  283.589996  283.600006  560645.0   
[2022-04-13 11:00:00, 2022-04-13 11:15:00)  283.614990  285.300110  758498.0   
[2022-04-13 11:15:00, 2022-04-13 11:30:00)  284.890015  285.559998  623103.0   

symbol                                           BTC-USD                \
                                                    open          high   
[2022-04-13 10:30:00, 2022-04-13 10:45:00)  40736.816406  40736.816406   
[2022-04-13 10:45:00, 2022-04-13 11:00:00)  40671.167969  40691.929688   
[2022-04-13 11:00:00, 2022-04-13 11:15:00)  40660.738281  40885.968750   
[2022-04-13 11:15:00, 2022-04-13 11:30:00)  40897.105469  41252.726562   

symbol                                                                  \
                                                     low         close   
[2022-04-13 10:30:00, 2022-04-13 10:45:00)  40499.980469  40619.902344   
[2022-04-13 10:45:00, 2022-04-13 11:00:00)  40618.472656  40657.050781   
[2022-04-13 11:00:00, 2022-04-13 11:15:00)  40660.738281  40877.394531   
[2022-04-13 11:15:00, 2022-04-13 11:30:00)  40891.882812  41252.726562   

symbol                                                    
                                                  volume  
[2022-04-13 10:30:00, 2022-04-13 10:45:00)  4.158497e+10  
[2022-04-13 10:45:00, 2022-04-13 11:00:00)  1.867203e+08  
[2022-04-13 11:00:00, 2022-04-13 11:15:00)  3.322307e+08  
[2022-04-13 11:15:00, 2022-04-13 11:30:00)  6.633042e+08

...with this where the input's timezone is assumed as UTC (the timezone associated with Bitcoin)...

In [81]:
df = prices.get("15T", start, hours=1, tzin="BTC-USD", tzout="MSFT")
df

symbol                                     MSFT                        \
                                           open high low close volume   
[2022-04-13 06:30:00, 2022-04-13 06:45:00)  NaN  NaN NaN   NaN    NaN   
[2022-04-13 06:45:00, 2022-04-13 07:00:00)  NaN  NaN NaN   NaN    NaN   
[2022-04-13 07:00:00, 2022-04-13 07:15:00)  NaN  NaN NaN   NaN    NaN   
[2022-04-13 07:15:00, 2022-04-13 07:30:00)  NaN  NaN NaN   NaN    NaN   

symbol                                           BTC-USD                \
                                                    open          high   
[2022-04-13 06:30:00, 2022-04-13 06:45:00)  39899.207031  39923.304688   
[2022-04-13 06:45:00, 2022-04-13 07:00:00)  39893.808594  39930.082031   
[2022-04-13 07:00:00, 2022-04-13 07:15:00)  39831.414062  39915.304688   
[2022-04-13 07:15:00, 2022-04-13 07:30:00)  39899.246094  39984.316406   

symbol                                                                  \
                                                     low         close   
[2022-04-13 06:30:00, 2022-04-13 06:45:00)  39881.054688  39901.187500   
[2022-04-13 06:45:00, 2022-04-13 07:00:00)  39825.808594  39834.921875   
[2022-04-13 07:00:00, 2022-04-13 07:15:00)  39831.414062  39904.425781   
[2022-04-13 07:15:00, 2022-04-13 07:30:00)  39899.246094  39984.316406   

symbol                                                    
                                                  volume  
[2022-04-13 06:30:00, 2022-04-13 06:45:00)  1.421128e+08  
[2022-04-13 06:45:00, 2022-04-13 07:00:00)  2.760463e+10  
[2022-04-13 07:00:00, 2022-04-13 07:15:00)  2.014214e+10  
[2022-04-13 07:15:00, 2022-04-13 07:30:00)  0.000000e+00

Note: `tzout` can also take a symbol and, for the purpose of comparison, in both cases the output was set to the same timezone as the NYSE.

In [82]:
df.pt.tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

## `add_a_row`

Run the following cell to instantiate a price object for this last part of the tutorial.

In [83]:
prices = PricesYahoo("MSFT")  # prices for Microsoft only

The `add_a_row` parameter can be used to include the row immediately prior to the start of the evaluated period.

Compare...

In [84]:
prices.get("D", end="2022-01-24", days=3)

symbol            MSFT                                              
                  open        high         low       close    volume
2022-01-20  309.070007  311.649994  301.140015  301.600006  35380700
2022-01-21  302.690002  304.109985  295.609985  296.029999  57984400
2022-01-24  292.200012  297.109985  276.049988  296.369995  85731500

with...

In [85]:
df = prices.get(
    "D", end="2022-01-24", days=3, add_a_row=True, lose_single_symbol=True
)
df

,open,high,low,close,volume
2022-01-19,306.290009,313.910004,302.700012,303.329987,45933900
2022-01-20,309.070007,311.649994,301.140015,301.600006,35380700
2022-01-21,302.690002,304.109985,295.609985,296.029999,57984400
2022-01-24,292.200012,297.109985,276.049988,296.369995,85731500


This option is useful to evalute the price change over the period based on the close price.

In [86]:
pct_chg = 100 * (df.iloc[-1].close - df.iloc[0].close) / df.iloc[0].close
round(pct_chg, 2)

-2.29

The `add_a_row` option works for all intervals and bounds...

In [87]:
start = xnys.session_open(start_reg_T1).astimezone(xnys.tz)
start

Timestamp('2022-04-13 09:30:00-0400', tz='America/New_York')

In [88]:
prices.get("10T", start, minutes=30, add_a_row=True)

symbol                                            MSFT              \
                                                  open        high   
[2022-04-12 15:50:00, 2022-04-12 16:00:00)  282.190002  282.579987   
[2022-04-13 09:30:00, 2022-04-13 09:40:00)  282.049988  282.619995   
[2022-04-13 09:40:00, 2022-04-13 09:50:00)  281.769989  283.859985   
[2022-04-13 09:50:00, 2022-04-13 10:00:00)  283.230011  283.579193   

symbol                                                                         
                                                   low       close     volume  
[2022-04-12 15:50:00, 2022-04-12 16:00:00)  281.750000  281.980011  1613136.0  
[2022-04-13 09:30:00, 2022-04-13 09:40:00)  281.600006  281.750000  1502798.0  
[2022-04-13 09:40:00, 2022-04-13 09:50:00)  281.299988  283.250000  1185836.0  
[2022-04-13 09:50:00, 2022-04-13 10:00:00)  282.170013  282.760010   670237.0